# 시간 데이터 다루기

출처: [Everything you can do with a time series(kaggle)](https://www.kaggle.com/kernels/scriptcontent/4882227/notebook)

## Preparation

### Package Import

In [ ]:
# Importing libraries
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
# Above is a special style template for matplotlib, highly useful for visualizing time series data
%matplotlib inline

### Data Load

In [ ]:
google = pd.read_csv('./data/stock-time-series-20050101-to-20171231/GOOGL_2006-01-01_to_2018-01-01.csv')
google.head()

## datetime 다루기

### 시간 데이터로 바꾸기

In [ ]:
google.dtypes

Date 컬럼의 타입이 object(문자열)로 되어 있어서 `datetime` 타입으로 바꾼다.

In [ ]:
google["Date"] = pd.to_datetime(google["Date"])

In [ ]:
google.dtypes

In [ ]:
google.head()

#### \[연습문제\] format 인수 이용하기

In [ ]:
google["Date_Korea"] = google["Date"].dt.year.astype(str) + "월 " \
    + google["Date"].dt.month.astype(str) + "월 " \
    + google["Date"].dt.day.astype(str) + "일"

In [ ]:
google.head()

위의 명령을 실행하면 `Date_Korea`라는 컬럼이 생성된다. 이 컬럼에 `pd.to_datetime` 함수와 format argument를 사용해서 datetime 타입으로 변환하여 Date2라는 컬럼에 저장하는 코드를 작성하라

In [ ]:
google["Date2"] = pd.to_datetime(google["Date_Korea"], format="%Y월 %m월 %d일")

In [ ]:
google.dtypes

### Indexing

> (주의) \[\]를 이용해 구간을 추출하기 위해서는 datetime 타입을 가진 column이 인덱스가 되어야 함

In [ ]:
google.set_index("Date",inplace=True)

In [ ]:
google.head()

In [ ]:
# 2008년부터 2010년 까지 데이터만 추출하기
google['2008':'2010'].head()

In [ ]:
# 2008년 1월부터 2009년 6월년 까지 데이터만 추출하기
google['2008-01':'2009-06']

In [ ]:
# 2008년 1월부터 2009년 6월년 까지 데이터만 추출하기(다른 포맷, 알아서 인식함)
google['2008/01':'2009/06']

#### \[연습문제\] indexing 이용하기

2006년 1월 3일부터 2006년 6월 30일 까지의 시가를 시계열그림으로 나타내라(`Date` 컬럼과 `Open` 컬럼 이용)

In [ ]:
google["2006/1/3":"2006/6/30"]["Open"].plot()
plt.show()

In [ ]:
# 인덱스 리셋
google.reset_index(inplace=True)

## `.dt` Accessor 

`.dt` accessor를 이용하면 datetime 타입의 컬럼에서 필요한 값을 쉽게 구할 수 있다.

In [ ]:
# 월 구하기
google["Date"].dt.month.head()

In [ ]:
# 요일 구하기
# 0 (월요일) - 6 (일요일)
google["Date"].dt.dayofweek.head()

#### \[연습문제\] year 컬럼 만들기

`Date` 컬럼에서 연도를 추출해 `Year` 컬럼을 만든다

In [ ]:
google["Year"] = google["Date"].dt.year

#### \[연습문제\] 연도별 시가 평균 구하기

`Year` 컬럼과 `groupby`를 이용해 연도별 평균 시가를 구한다.

In [ ]:
google.groupby("Year")["Open"].mean()

## `DateOffset`

날짜에 시간을 더하거나 뺄때는 `DateOffset`을 이용한다.

In [ ]:
dates = google["Date"].head()

In [ ]:
# 기존 데이터
dates.head()

In [ ]:
# 하루 뒤로 옮기기
dates + pd.DateOffset(days=1)

In [ ]:
# 하루 전으로 옮기기
dates + pd.DateOffset(days=-1)

시간도 `DateOffset`으로 바꿀 수 있다.

In [ ]:
# 한 시간 뒤로 옮기기 (날짜까지 자동으로 계산된 것도 확인하기)
dates + pd.DateOffset(hours=-1)

## `shift` 이용하기

datetime인 컬럼이 인덱스가 되어야 사용할 수 있음

In [ ]:
google.set_index("Date", inplace=True)

In [ ]:
google.head()

In [ ]:
# 데이터를 한 시점 앞으로 이동시키기
# 모든 데이터가 뒤 시점으로 밀림
google.shift(1).head()

In [ ]:
# 데이터를 한 시점 뒤로 이동시키기
# 모든 데이터가 이전 시점으로 밀림
google.shift(-1).head()

In [ ]:
# 데이터를 한 시점 뒤로 이동시키기
# 하루단위가 아닌 다른 단위로 이동시키고 싶을 때 freq 이용
# Date2 컬럼이 shift하기 전의 원래 시점
google.shift(-1, freq="M").head()

이를 이용해 이전 시점과의 차이를 볼 때 유용하게 사용할 수 있다.

In [ ]:
google["OpenDiff"] = google["Open"] - google["Open"].shift(-1)

In [ ]:
google["OpenDiff"].head()

시가(`Open`)와 전일 대비 시가(`OpenDiff`) 차이에 대한 시계열 그래프

In [ ]:
google["2007"][["Open","OpenDiff"]].plot(figsize=[20,10])
plt.show()

## Windowing

예를 들어 12개 단위로 평균, 합을 계산하고 싶을 때 12개씩 데이터를 끊어서 (windowing) 합, 평균, 표준편차 등을 구할 수 있다.

`rolling` 메소드를 이용한다.

In [ ]:
google["Open"].rolling(window=12).mean()

### \[ 연습문제\] 시가의 3일 이동평균에 대한 시계열 그림 그리기

`Open`컬럼과 `rolling`을 이용해 3일 이동평균에 대한 시계열 그림을 그려본다.

In [ ]:
google["Open"].rolling(window=3).mean().plot(figsize=(20,10))
plt.show()